In [ ]:
import dlib
import numpy as np
import moviepy.editor as mp
import cv2

# Load the video and audio files
video_path = "/content/TechNews 1562 __ IPL Final, OLA Prime Plus, Love scam, Motorola Edge 40, Premium TV Days Etc....mp4"
audio_path = "/content/output10.wav"


# Load the face detector and landmarks predictor
face_detector = dlib.get_frontal_face_detector()
landmarks_predictor = dlib.shape_predictor("/content/shape_predictor_68_face_landmarks.dat")

# Load the video clip
video_clip = mp.VideoFileClip(video_path)
audio_clip = mp.AudioFileClip(audio_path)
audio_clip = audio_clip.subclip(0, 67)
# Get the fps of the video
fps = video_clip.fps

# Function to extract landmarks from a frame
def get_landmarks(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_detector(gray)
    if faces:
        landmarks = landmarks_predictor(gray, faces[0])
        landmarks = np.array([[p.x, p.y] for p in landmarks.parts()])
        return landmarks
    else:
        return None

# Process video frames and synchronize with audio
new_frames = []
for idx, frame in enumerate(video_clip.iter_frames(fps=fps, dtype="uint8")):
    landmarks = get_landmarks(frame)
    if landmarks is not None:
        # Calculate audio amplitude at the current frame
        start_time = idx / fps
        end_time = (idx + 1) / fps
        audio_amplitude = np.mean(audio_clip.subclip(start_time, end_time).to_soundarray())

        # Modify lip landmarks based on audio amplitude (example: move lip landmarks up/down)
        amplitude_modification = 10 * audio_amplitude  # Adjust the multiplier based on the effect you want
        processed_landmarks = landmarks + amplitude_modification

        # Create a new frame with modified landmarks
        processed_frame = frame.copy()
        for landmark in processed_landmarks:
            x, y = landmark.astype(int)
            cv2.circle(processed_frame, (x, y), 3, (0, 255, 0), -1)

        # Add processed frame with modified landmarks
        new_frames.append(processed_frame)
    else:
        new_frames.append(frame)

# Create a new video with processed frames and original audio
processed_video = mp.ImageSequenceClip(new_frames, fps=fps)
processed_video = processed_video.set_audio(audio_clip)

# Save the final lip-synced video
processed_video.write_videofile("lip_synced_video.mp4", codec="libx264", audio_codec="aac")
